# Demo (structuring)

Injection of first order logic rules into a neural network for iris classification task via structuring.

Some imports.

In [1]:
from psyki import StructuringInjector
from psyki.fol import Parser
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from test import get_mlp
from test import get_rules
from tensorflow.keras import Input, Model
from tensorflow.python.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam


Loading iris dataset and separation into train and test set.

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)

train, test = train_test_split(dataset, test_size=0.5, random_state=0)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


Import FOL rules:

- PL <= 2.28 <- X = setosa
- PL > 2.28 ^ PW <= 1.64 <- X = versicolor
- PL > 2.28 ^ PW > 1.64 <- X = virginica

In [3]:
features_mapping = {
    'SL': 0,
    'SW': 1,
    'PL': 2,
    'PW': 3,
}

parser = Parser.default_parser()
iris_rules = get_rules('iris')

Injection of fuzzy logic function derived from FOL rules into a neural network.

In [4]:
input_features = Input((4,), name='Input')
injector = StructuringInjector(parser)
network = get_mlp(input=input_features, output=3, layers=3, neurons=16, activation_function='relu',
                  last_activation_function='softmax')
main_network = Model(input_features, network).layers[-2].output
model = injector.inject(iris_rules, input_features, main_network, 3, 'softmax', features_mapping)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 1)            5           Input[0][0]                      
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           Input[0][0]                      
____________________________________________________________________________________________

Training.

In [5]:
train_x, train_y = train.iloc[:,:-1], train.iloc[:,-1]
model.fit(train_x, train_y, verbose=1, batch_size=4, epochs=50)

Epoch 1/50


2022-02-08 14:39:04.143086: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-08 14:39:04.143234: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


19/19 [==============================] - 0s 471us/step - loss: 1.2792 - accuracy: 0.2667
Epoch 2/50
19/19 [==============================] - 0s 390us/step - loss: 1.0322 - accuracy: 0.3733
Epoch 3/50
19/19 [==============================] - 0s 348us/step - loss: 0.9134 - accuracy: 0.4400
Epoch 4/50
19/19 [==============================] - 0s 348us/step - loss: 0.8379 - accuracy: 0.6933
Epoch 5/50
19/19 [==============================] - 0s 365us/step - loss: 0.7931 - accuracy: 0.7333
Epoch 6/50
19/19 [==============================] - 0s 336us/step - loss: 0.7521 - accuracy: 0.7333

Evaluation.

In [6]:
test_x, test_y = test.iloc[:,:-1], test.iloc[:,-1]
model.evaluate(test_x, test_y)

3/3 [==============================] - 0s 701us/step - loss: 0.2809 - accuracy: 0.9600


[0.2808704376220703, 0.9599999785423279]